In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
from pprint import pprint as pp
import json
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Line, LinearAxis, Range1d
import pandas_bokeh
import os

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
conn = sqlite3.connect('../db/11_Jun.db')

In [4]:
ndvi_df = pd.read_sql("select * from ndvi", conn, parse_dates=['date_time'])
ndvi_sorted = ndvi_df.sort_values('date_time')
evi_df = pd.read_sql("select * from evi", conn, parse_dates=['date_time'])
evi_sorted = evi_df.sort_values('date_time')
lai_df = pd.read_sql("select * from lai", conn, parse_dates=['date_time'])
lai_sorted = lai_df.sort_values('date_time')

In [20]:
evi_df.head(30)

,georegion,max,min,mean,median,std,var,pixels,25_percent,50_percent,75_percent,date_time,farm_visible,vis_moisture,e18,hay_field,cedar_hill
0,Farm boundary,210.000000,-0.990991,0.710822,0.065358,8.516522,72.531154,2154,-0.056763,0.065358,0.263248,2020-05-25 16:39:00,0,1,0,0,0
1,Paddock 1,0.570033,-0.099338,0.115860,0.133229,0.137404,0.018880,45,0.000000,0.133229,0.173913,2020-05-25 16:39:00,0,1,0,0,0
2,Paddock 2,1.443662,-0.113475,0.163901,0.092919,0.250351,0.062676,78,0.016722,0.092919,0.183986,2020-05-25 16:39:00,0,1,0,0,0
3,Paddock 3,0.750000,0.091463,0.376877,0.342809,0.161694,0.026145,43,0.293620,0.342809,0.468227,2020-05-25 16:39:00,0,1,0,0,0
4,Paddock 4,210.000000,0.000000,7.423738,0.551802,36.396852,1324.730857,64,0.304477,0.551802,1.201550,2020-05-25 16:39:00,0,1,0,0,0
5,Paddock 5,0.468750,0.000000,0.103394,0.086806,0.079905,0.006385,63,0.042614,0.086806,0.150665,2020-05-25 16:39:00,0,1,0,0,0
6,Paddock 6,0.652655,-0.021834,0.169340,0.143165,0.139896,0.019571,68,0.076060,0.143165,0.203016,2020-05-25 16:39:00,0,1,0,0,0
7,Paddock 7,5.853659,-0.106237,0.423116,0.133971,1.038894,1.079300,61,0.030120,0.133971,0.305085,2020-05-25 16:39:00,0,1,0,0,0
8,Paddock 8,0.677570,-0.096717,0.096384,0.057252,0.161802,0.026180,61,-0.032293,0.057252,0.183544,2020-05-25 16:39:00,0,1,0,0,0
9,Paddock 9,51.000000,-0.719064,1.736664,0.082210,7.354535,54.089181,48,-0.042847,0.082210,0.486002,2020-05-25 16:39:00,0,1,0,0,0


In [6]:
bound_evi = evi_sorted[(evi_sorted['georegion'] == 'Farm boundary') & (evi_sorted['vis_moisture'] == 0)]
bound_ndvi = ndvi_sorted[(ndvi_sorted['georegion'] == 'Farm boundary') & (ndvi_sorted['vis_moisture'] == 0)]
bound_lai = lai_sorted[(lai_sorted['georegion'] == 'Farm boundary') & (lai_sorted['vis_moisture'] == 0)]

In [34]:
bound_lai.head()

,georegion,max,min,mean,median,std,var,pixels,25_percent,50_percent,75_percent,date_time,farm_visible,vis_moisture,e18,hay_field,cedar_hill
1736,Farm boundary,3.000139,0.546205,1.659675,1.632700,0.326041,0.106303,568,1.438162,1.632700,1.867938,2019-04-09 16:48:00,1,0,1,1,1
0,Farm boundary,2.477568,0.712379,1.647225,1.679992,0.335318,0.112438,568,1.361309,1.679992,1.899972,2019-04-16 16:38:00,1,0,1,1,1
1316,Farm boundary,2.917079,0.829056,2.004332,2.030180,0.414038,0.171428,568,1.668260,2.030180,2.342476,2019-04-19 16:48:00,1,0,1,1,1
1540,Farm boundary,3.417554,0.851349,2.395017,2.507659,0.552238,0.304967,568,1.966011,2.507659,2.853257,2019-04-26 16:38:00,1,0,1,1,1
616,Farm boundary,3.576668,1.028668,2.953631,2.999650,0.348976,0.121784,568,2.804371,2.999650,3.195860,2019-05-31 16:39:00,1,0,1,1,1


In [13]:
evi_source = ColumnDataSource(bound_evi)
ndvi_source = ColumnDataSource(bound_ndvi)
lai_source = ColumnDataSource(bound_lai)
precip_source = ColumnDataSource(valid_precip)

p = figure(x_axis_type="datetime", y_range=(0,1),
          title='Index data',
          x_axis_label='Date',
          y_axis_label='Index value')
p.line('date_time', 'mean', source=evi_source, legend_label='EVI', color='green', line_width=2)
p.line('date_time', 'mean', source=ndvi_source, legend_label='NDVI', color='red', line_width=2)
p.line('DATE', 'SMA', source=precip_source, legend_label='Rain')

p.extra_y_ranges = {"lai": Range1d(start=0, end=5)}
p.line('date_time', 'mean', source=lai_source, legend_label='LAI', color="orange", line_width=2, y_range_name='lai')
p.add_layout(LinearAxis(y_range_name="lai"), 'right')

show(p)

In [7]:
precip_df = pd.read_csv('../../data/Jan 19 - May 20 precip.csv', parse_dates=['DATE'])
precip_df

,STATION,NAME,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES
0,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-01,0.00,"T,,W,2400",0.0,"T,,W"
1,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-02,0.00,"T,,W,2400",0.0,"T,,W"
2,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-03,0.00,",,W,2400",0.0,",,W"
3,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-04,0.00,",,W,2400",0.0,",,W"
4,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-05,0.00,",,W,2400",0.0,",,W"
...,...,...,...,...,...,...,...
494,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-09,0.00,"T,,D,2400",0.0,",,D"
495,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-10,0.25,",,D,2400",0.0,",,D"
496,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-11,0.00,"T,,D,2400",0.0,",,D"
497,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-12,0.00,",,D,2400",0.0,",,D"


In [8]:
precip_df['SMA'] = precip_df.loc[:,['PRCP']].rolling(window=21).mean()
valid_precip = precip_df[precip_df['SMA'] >= 0]

In [18]:
conn.close()